# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2024-04-28 07:44:57 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


# Initial Solver
Run Installation First


In [1]:
import Pkg; Pkg.add("CSV")
import Pkg; Pkg.add("Tables")
import Pkg; Pkg.add("JuMP")
import Pkg; Pkg.add("HiGHS")

import CSV
import Tables
import JuMP
import HiGHS
import JuMP.Containers
using CSV
using Tables
using JuMP
using HiGHS
using JuMP.Containers

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed TableTraits ───────────────── v1.0.1
   Installed PooledArrays ──────────────── v1.4.3
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed SentinelArrays ────────────── v1.4.1
   Installed InlineStrings ─────────────── v1.4.0
   Installed Tables ────────────────────── v1.11.1
   Installed DataAPI ───────────────────── v1.16.0
   Installed DataValueInterfaces ───────── v1.0.0
   Installed OrderedCollections ────────── v1.6.3
   Installed WeakRefStrings ────────────── v1.4.2
   Installed FilePathsBase ─────────────── v0.9.21
   Installed Compat ────────────────────── v4.14.0
   Installed WorkerUtilities ───────────── v1.6.1
   Installed CodecZlib ─────────────────── v0.7.4
   Installed TranscodingStreams ────────── v0.10.7
   Installed CSV ───────────────────────── v0.10.14
    Updating `~/.julia/environments/v1.8/Project.toml`
  [336ed68f] + CSV v0.10.14
    Updating `~/.

In [5]:
input_evacuee_matrix = CSV.File("input_evacuee_matrix.csv") |> Tables.matrix
input_shelter_matrix = CSV.File("input_shelter_matrix.csv") |> Tables.matrix

evacuee_names = input_evacuee_matrix[1:990,1]
shelter_names = input_shelter_matrix[:,1]

187-element Vector{Any}:
 "桜丘小学校"
 "中山中学校"
 "中山小学校"
 "北仙台中学校"
 "台原小学校"
 "旭丘小学校"
 "荒巻小学校"
 "国見小学校"
 "三条中学校"
 "通町小学校"
 "小松島小学校"
 "八幡小学校"
 "木町通小学校"
 ⋮
 "将監東中学校"
 "将監中学校"
 "将監西小学校"
 "高森東小学校"
 "長命ケ丘小学校"
 "南中山中学校"
 "寺岡小学校"
 "住吉台中学校"
 "根白石中学校"
 "館中学校"
 "仙台商業高等学校"
 "市名坂小学校"

In [ ]:
full_cost_matrix = CSV.File("final_cost_matrix.csv") |> Tables.matrix

edited_cost_matrix = full_cost_matrix[:,2:end]

full_cost_array = Containers.DenseAxisArray(edited_cost_matrix, 1:990, 1:187)

shelter_numbers = input_shelter_matrix[1:end,2]

evacuee_numbers = input_evacuee_matrix[1:end,2]

function solve_transportation_problem(data::Containers.DenseAxisArray,shelter_numbers,evacuee_numbers)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    #O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == evacuee_numbers[o])
    @constraint(model, [d in D], sum(x[:, d]) <= shelter_numbers[d])
    optimize!(model)


    # Construct the solution matrix
    solution_matrix = zeros(length(O), length(D))  # Initialize with zeros

    for (i, o) in enumerate(O)
        for (j, d) in enumerate(D)
            solution_matrix[i, j] = value(x[o, d])
        end
    end

    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return solution_matrix
end

print("\nSolution:\n")
solution_matrix = solve_transportation_problem(full_cost_array, shelter_numbers, evacuee_numbers)
println(solution_matrix)

CSV.write("sendai_solution_matrix.csv", Tables.table(solution_matrix))


In [7]:
#NOT DONE Total Cost Calculation
solution_cost = full_cost_array.data[1:end-1,1:end-1]
matching_indicies = findall(x-> x == ".", solution_cost)
solution_cost[matching_indicies] .= 0
#solution_matrix .* solution_cost
total_cost_matrix = solution_cost.*solution_matrix
running_total = sum(total_cost_matrix)
initial_running_total = running_total
print("Average distance per evacuee: ")
println(initial_running_total/sum(evacuee_numbers))

Average distance per evacuee: 0.9297263176521238


# Testing

In [8]:
function is_evacuee_numbers_matching(solution_matrix,evacuee_numbers)
  column_wise_solution_sum = sum(solution_matrix, dims=2)
  int_column_wise_solution_sum = Int.(column_wise_solution_sum)
  evacuees_comparison = int_column_wise_solution_sum .== evacuee_numbers
  return prod(evacuees_comparison)
end

is_evacuee_numbers_matching(solution_matrix,evacuee_numbers)

true

In [9]:
function is_shelter_not_overflowing(solution_matrix,shelter_numbers)
  row_wise_solution_sum = sum(solution_matrix, dims=1)
  int_row_wise_solution_sum = Int.(row_wise_solution_sum)
  shelter_comparison = int_row_wise_solution_sum[1,:] .<= shelter_numbers
  return prod(shelter_comparison)
end

is_shelter_not_overflowing(solution_matrix,shelter_numbers)

true

In [10]:
#Evacuee = shelter_max +1 overload check
print("Shelter Total Capacity: " )
println(sum(shelter_numbers))
print("Evacuee Numbers: " )
println(sum(evacuee_numbers))

shelter_evacuee_difference = sum(shelter_numbers) - sum(evacuee_numbers)
overload_evacuee_numbers = deepcopy(evacuee_numbers)
overload_evacuee_numbers[1] += shelter_evacuee_difference +1

print("Overload Evacuee Numbers: ")
println(sum(overload_evacuee_numbers))

overload_solution_matrix = solve_transportation_problem(full_cost_array, shelter_numbers, overload_evacuee_numbers)
println(overload_solution_matrix)


Shelter Total Capacity: 1301750
Evacuee Numbers: 1096344
Overload Evacuee Numbers: 1301751


LoadError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [11]:
full_cost_array = Containers.DenseAxisArray(edited_cost_matrix, 1:990, 1:187)
println(size(full_cost_array))
#println(num_rows)
#println(num_cols)
#println(size(edited_cost_matrix))

(991, 188)


# Derivative Calculation

In [43]:
full_cost_matrix = CSV.File("final_cost_matrix.csv") |> Tables.matrix

edited_cost_matrix = full_cost_matrix[:,2:end]

full_cost_array = Containers.DenseAxisArray(edited_cost_matrix, 1:990, 1:187)

maximum_distance_variable = 1000

num_rows, num_cols = axes(full_cost_array)
testing_counter = 0
for i in num_rows
    for j in num_cols
        if full_cost_array[i,j]>= maximum_distance_variable
            testing_counter+=1
        end
    end
end
print("total calculations: ")
println((size(num_rows)[1]*(size(num_cols)[1])))
print("reduced calculations: ")
println(testing_counter)
print("new total calculations: ")
println(((size(num_rows)[1]*(size(num_cols)[1]))-testing_counter))

total calculations: 185130
reduced calculations: 0
new total calculations: 185130


In [44]:
edited_cost_matrix = full_cost_matrix[:,2:end]

maximum_distance_full_cost_array = Containers.DenseAxisArray(edited_cost_matrix, 1:991, 1:188)
num_rows, num_cols = axes(maximum_distance_full_cost_array)
num_rows = num_rows[1:end-1]
num_cols = num_cols[1:end-1]
for i in num_rows
    for j in num_cols
      try
        if maximum_distance_full_cost_array[i,j]>= maximum_distance_variable
           maximum_distance_full_cost_array[i,j] = "."
        end
      catch e
        println(i)
        println(j)
        println(e)
      end
    end
end
CSV.write("maximum_distance_full_cost_array.csv", Tables.table(maximum_distance_full_cost_array))

"maximum_distance_full_cost_array.csv"

In [45]:
#DONE CHECK IF EACH DISASTER LOCATION HAS A SHELTER
maximum_distance_full_cost_array_analysis = CSV.File("maximum_distance_full_cost_array.csv") |> Tables.matrix
num_rows, num_cols = axes(maximum_distance_full_cost_array)
num_rows = num_rows[1:end-1]
num_cols = num_cols[1:end-1]
for i in num_rows
  for j in num_cols
    if maximum_distance_full_cost_array_analysis[i,j] == "."
      maximum_distance_full_cost_array_analysis[i,j] = 0
      else
      maximum_distance_full_cost_array_analysis[i,j] = 1
    end
  end
end
sum_maximum_distance_full_cost_array_analysis = sum(maximum_distance_full_cost_array_analysis[1:end-1,1:end-1], dims = 2)
print("Total disaster locations with possible route: ")
print(sum(clamp.(sum_maximum_distance_full_cost_array_analysis, 0, 1)))
print("/")
print(size(num_rows)[1])

Total disaster locations with possible route: 990.0/990

In [65]:
#THIS DERIVATIVE WORKS 4/28
maximum_distance_full_cost_array = CSV.File("maximum_distance_full_cost_array.csv") |> Tables.matrix
data = maximum_distance_full_cost_array[1:end-1,1:end-1]
epsilon = 1
num_rows, num_cols = axes(data)
derivative_matrix = zeros(Float64, length(num_rows), length(num_cols))

for (i, o) in enumerate(num_rows[1:3]), (j, d) in enumerate(num_cols[1:3])
  original_cost = data[o, d]
    if original_cost != "."
      #original_cost = parse(Float64, original_cost)
      println(i)
      println(j)
      # Increase the cost slightly and re-solve
      data[o, d] = original_cost + epsilon

      increased_cost_model = Model(HiGHS.Optimizer)
      set_silent(increased_cost_model)
      @variable(increased_cost_model, x[ii in num_rows, jj in num_cols] >= 0)

      for ii in num_rows, jj in num_cols
        if data[ii, jj] == "."
          fix(x[ii, jj], 0.0; force = true)
        end
      end

      @objective(
        increased_cost_model,
        Min,
        sum(data[ii, jj] * x[ii, jj] for ii in num_rows, jj in num_cols if data[ii, jj] isa Number),
      )

      @constraint(increased_cost_model, [ii in num_rows], sum(x[ii, :]) == evacuee_numbers[ii])
      @constraint(increased_cost_model, [jj in num_cols], sum(x[:, jj]) <= shelter_numbers[jj])

      optimize!(increased_cost_model)

      increased_cost_objective = objective_value(increased_cost_model)
      # Decrease the cost slightly and re-solve
      data[o, d] = original_cost - epsilon

      decreased_cost_model = Model(HiGHS.Optimizer)
      set_silent(decreased_cost_model)
      @variable(decreased_cost_model, x[ii in num_rows, jj in num_cols] >= 0)

      for ii in num_rows, jj in num_cols
          if data[ii, jj] == "."
              fix(x[ii, jj], 0.0; force = true)
          end
      end

      @objective(
          decreased_cost_model,
          Min,
          sum(data[ii, jj] * x[ii, jj] for ii in num_rows, jj in num_cols if data[ii, jj] isa Number),
      )
      @constraint(decreased_cost_model, [ii in num_rows], sum(x[ii, :]) == evacuee_numbers[ii])
      @constraint(decreased_cost_model, [jj in num_cols], sum(x[:, jj]) <= shelter_numbers[jj])
      optimize!(decreased_cost_model)
      decreased_cost_objective = objective_value(decreased_cost_model)
      # Calculate the numerical derivative
      derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
      derivative_matrix[i, j] = derivative
      # Reset the original cost
      data[o, d] = original_cost
    end
end
println(derivative_matrix)

1
1


LoadError: InterruptException:

In [ ]:
#backup of
#working on this one
maximum_distance_full_cost_array = CSV.File("maximum_distance_full_cost_array.csv") |> Tables.matrix
data = maximum_distance_full_cost_array[1:end-1,1:end-1]
epsilon = 1
num_rows, num_cols = axes(data)
num_rows = num_rows[1:end-1]
num_cols = num_cols[1:end-1]
derivative_matrix = zeros(Float64, length(num_rows), length(num_cols))
O, D = axes(data)

for (i, o) in enumerate(num_rows), (j, d) in enumerate(num_cols)
  original_cost = data[o, d]
    if original_cost != "."
      original_cost = parse(Float64, original_cost)
      println("hi")
      # Increase the cost slightly and re-solve
      data[o, d] = original_cost + epsilon
      increased_cost_model = Model(HiGHS.Optimizer)
      set_silent(increased_cost_model)
      @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

      for ii in O, jj in D
        if data[ii, jj] == "."
          fix(x[ii, jj], 0.0; force = true)
        end
      end
      @objective(
        increased_cost_model,
        Min,
        sum(data[ii, jj] * x[ii, jj] for ii in num_rows, jj in num_cols if data[ii, jj] isa Number),
      )
      @constraint(increased_cost_model, [ii in num_rows], sum(x[ii, :]) == data[ii, evacuee_numbers])
      @constraint(increased_cost_model, [jj in num_cols], sum(x[:, jj]) <= data[shelter_numbers, jj])
      optimize!(increased_cost_model)
      increased_cost_objective = objective_value(increased_cost_model)
      # Decrease the cost slightly and re-solve
      data[o, d] = original_cost - epsilon
      decreased_cost_model = Model(HiGHS.Optimizer)
      set_silent(decreased_cost_model)
      @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

      for ii in O, jj in D
          if data[ii, jj] == "."
              fix(x[ii, jj], 0.0; force = true)
          end
      end
      @objective(
          decreased_cost_model,
          Min,
          sum(data[ii, jj] * x[ii, jj] for ii in num_rows, jj in num_cols if data[ii, jj] isa Number),
      )
      @constraint(decreased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, evacuee_numbers])
      @constraint(decreased_cost_model, [jj in D], sum(x[:, jj]) <= data[shelter_numbers, jj])
      optimize!(decreased_cost_model)
      decreased_cost_objective = objective_value(decreased_cost_model)
      # Calculate the numerical derivative
      derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
      derivative_matrix[i, j] = derivative
      # Reset the original cost
      data[o, d] = original_cost
    end
end

In [ ]:
#Derivative Calculation (Pathway Cost)

function calculate_numerical_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, shelter_numbers, evacuee_numbers, epsilon::Float64 = 1)
    O, D = axes(data)
    #O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_cost = data[o, d]

        if original_cost isa Number
            # Increase the cost slightly and re-solve
            data[o, d] = original_cost + epsilon
            increased_cost_model = Model(HiGHS.Optimizer)
            set_silent(increased_cost_model)
            @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, evacuee_numbers])
            @constraint(increased_cost_model, [jj in D], sum(x[:, jj]) <= data[shelter_numbers, jj])

            optimize!(increased_cost_model)
            increased_cost_objective = objective_value(increased_cost_model)

            # Decrease the cost slightly and re-solve
            data[o, d] = original_cost - epsilon
            decreased_cost_model = Model(HiGHS.Optimizer)
            set_silent(decreased_cost_model)
            @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, evacuee_numbers])
            @constraint(decreased_cost_model, [jj in D], sum(x[:, jj]) <= data[shelter_numbers, jj])

            optimize!(decreased_cost_model)
            decreased_cost_objective = objective_value(decreased_cost_model)

            # Calculate the numerical derivative
            derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original cost
            data[o, d] = original_cost
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(full_cost_array, shelter_numbers, evacuee_numbers, epsilon)
println("\nNumerical Derivative Matrix for Pathway:")
for row in eachrow(derivative_matrix)
    println(row)
end

# Derivative Calculation (Shelter Capacity)

function calculate_numerical_capacity_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, shelter_numbers, evacuee_numbers, epsilon::Float64 = 1000)
    O, D = axes(data)
    #O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_capacity = data[shelter_numbers, d]

        if original_capacity isa Number
            # Increase the capacity slightly and re-solve
            data[shelter_numbers, d] = original_capacity + epsilon
            increased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(increased_capacity_model)
            @variable(increased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_capacity_model, [ii in O], sum(x[ii, :]) == data[ii, evacuee_numbers])
            @constraint(increased_capacity_model, [jj in D], sum(x[:, jj]) <= data[shelter_numbers, jj])

            optimize!(increased_capacity_model)
            increased_capacity_objective = objective_value(increased_capacity_model)

            # Decrease the capacity slightly and re-solve
            data[shelter_numbers, d] = original_capacity - epsilon
            decreased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(decreased_capacity_model)
            @variable(decreased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_capacity_model, [ii in O], sum(x[ii, :]) == data[ii, evacuee_numbers])
            @constraint(decreased_capacity_model, [jj in D], sum(x[:, jj]) <= data[shelter_numbers, jj])

            optimize!(decreased_capacity_model)
            decreased_capacity_objective = objective_value(decreased_capacity_model)

            # Calculate the numerical derivative
            derivative = (increased_capacity_objective - decreased_capacity_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original capacity
            data[shelter_numbers, d] = original_capacity
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix for shelter capacity changes
epsilon = 1000.0
capacity_derivative_matrix = calculate_numerical_capacity_derivative(full_cost_array, shelter_numbers, evacuee_numbers, epsilon)
println("\nNumerical Derivative Matrix for Shelter Capacity Changes:")
for row in eachrow(capacity_derivative_matrix)
    println(row)
end

In [18]:
# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(full_cost_array, shelter_numbers, evacuee_numbers, epsilon)
println("\nNumerical Derivative Matrix for Pathway:")
for row in eachrow(derivative_matrix)
    println(row)
end

# Derivative Calculation (Shelter Capacity)

function calculate_numerical_capacity_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, shelter_numbers, evacuee_numbers, epsilon::Float64 = 1000)
    O, D = axes(data)
    #O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_capacity = data[shelter_numbers, d]

        if original_capacity isa Number
            # Increase the capacity slightly and re-solve
            data[shelter_numbers, d] = original_capacity + epsilon
            increased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(increased_capacity_model)
            @variable(increased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_capacity_model, [ii in O], sum(x[ii, :]) == data[ii, evacuee_numbers])
            @constraint(increased_capacity_model, [jj in D], sum(x[:, jj]) <= data[shelter_numbers, jj])

            optimize!(increased_capacity_model)
            increased_capacity_objective = objective_value(increased_capacity_model)

            # Decrease the capacity slightly and re-solve
            data[shelter_numbers, d] = original_capacity - epsilon
            decreased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(decreased_capacity_model)
            @variable(decreased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_capacity_model, [ii in O], sum(x[ii, :]) == data[ii, evacuee_numbers])
            @constraint(decreased_capacity_model, [jj in D], sum(x[:, jj]) <= data[shelter_numbers, jj])

            optimize!(decreased_capacity_model)
            decreased_capacity_objective = objective_value(decreased_capacity_model)

            # Calculate the numerical derivative
            derivative = (increased_capacity_objective - decreased_capacity_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original capacity
            data[shelter_numbers, d] = original_capacity
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix for shelter capacity changes
epsilon = 1000.0
capacity_derivative_matrix = calculate_numerical_capacity_derivative(full_cost_array, shelter_numbers, evacuee_numbers, epsilon)
println("\nNumerical Derivative Matrix for Shelter Capacity Changes:")
for row in eachrow(capacity_derivative_matrix)
    println(row)
end

LoadError: KeyError: key Any[649, 741, 2617, 2619, 1836, 2088, 1604, 228, 121, 42  …  307, 0, 606, 704, 1364, 29, 146, 369, 1636, 1319] not found